Population vs. Median Home Prices
Linear Regression with Single Variable

Load and parse the data

In [ ]:
# Use the Spark CSV datasource with options specifying:
#  - First line of file is a header
#  - Automatically infer the schema of the data
data = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header="true", inferSchema="true")
data.cache()  # Cache data for faster reuse
data.count()

In [ ]:
display(data)

In [ ]:
data = data.dropna()  # drop rows with missing values
data.count()

In [ ]:
from pyspark.sql.functions import col

exprs = [col(column).alias(column.replace(' ', '_')) for column in data.columns]

vdata = data.select(*exprs).selectExpr("2014_Population_estimate as population", "2015_median_sales_price as label")
display(vdata)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

stages = []
assembler = VectorAssembler(inputCols=["population"], outputCol="features")
stages += [assembler]
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(vdata)
dataset = pipelineModel.transform(vdata)
# Keep relevant columns
selectedcols = ["features", "label"]
display(dataset.select(selectedcols))

Scatterplot of the data